In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *

In [ ]:
from diffusion_graph import read_graph

In [ ]:
G = read_graph(cache_dir+'diffusion_graph/pruned_graph_v2.tsv')
article_details = pd.read_csv(cache_dir+'article_details_v3.tsv', sep='\t')
tweet_details = pd.read_csv(cache_dir+'tweet_details_v1.tsv', sep='\t')

In [ ]:
def aggregate_attribute(attribute, tweets, tweet_details):
    if attribute == 'popularity':
        agg = 0 
        for t in tweets:
            agg += tweet_details[tweet_details['url'] == t][attribute].iloc[0]
        return agg
    

attribute = 'popularity'
article_details.sample(1)['url'].apply(lambda x: aggregate_attribute(attribute, G.predecessors(x), tweet_details))

--

In [ ]:
tweet_details.sample(1)['url'].apply(lambda x: x.split('/')[3])

In [ ]:
tweet_details.iloc[6952]['url']